In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/open.zip"

In [ ]:
# 사용 가능한 GPU 확인
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# GPU 사용 설정
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
sys.path.append(os.path.join(os.getcwd(), os.path.pardir, 'functions'))

from get_train_list import get_train_image_paths_and_encoding_labels_fn
from predict_and_test import get_predict_result_fn, get_test_csv_fn
from get_img_tensor_v2 import get_tensor_through_imgs_fn

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.data import Dataset, experimental
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from datetime import datetime
from PIL import ImageFile

In [3]:
# 학습용 이미지 경로와 라벨에 대한 각각의 리스트 생성하기
train_image_paths, encoding_labels = get_train_image_paths_and_encoding_labels_fn()

# ========= 학습 이미지 로드, 정규화, 사이즈 조정 =========

# 잘린 이미지에 대한 로드 여부 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

# EfficientNetB0()의 default_resolution
RESIZED_WIDTH = 224
RESIZED_HEIGHT = 224

train_image_tensor = get_tensor_through_imgs_fn(train_image_paths, RESIZED_WIDTH, RESIZED_HEIGHT)
print('train_image_tensor.shape :', train_image_tensor.shape)

# =======================================================

encoding_labels.shape : (3457, 19)


100%|██████████| 3457/3457 [00:58<00:00, 58.69it/s]


train_image_tensor.shape : (3457, 224, 224, 3)


In [4]:
# ============ EfficientNetB0() 사용해보기 ============

# 데이콘 리더 보드 점수 => 0.3871364552

# TRAIN_FOLDER_PATH = '/content/train/'
TRAIN_FOLDER_PATH = '../../open/train/'
OPEN_FOLDER_PATH = '../../open'

# EPOCHS = 10
EPOCHS = 1

train_ds = Dataset.from_tensor_slices(
    (train_image_tensor, encoding_labels))

train_ds_size = experimental.cardinality(train_ds).numpy()
print("Training data size:", train_ds_size)

train_ds = (train_ds
            .shuffle(buffer_size=train_ds_size)
            .batch(batch_size=32, drop_remainder=True))

model=Sequential()
model.add(EfficientNetB0(input_shape=(RESIZED_WIDTH, RESIZED_HEIGHT, 3), include_top=False, weights=None))
# model.add(Flatten())
model.add(Dense(19, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001),
               loss='categorical_crossentropy', metrics=['acc'])

model.summary()

train_history = model.fit(train_ds, epochs=EPOCHS)
loss = train_history.history['loss']
print('loss :', loss)

# 연월일_시간
now = datetime.today().strftime('%Y%m%d_%H%M%S')
model.save(f'model_{now}.h5')


Training data size: 3457
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 19)                1191699   
                                                                 
Total params: 5,241,270
Trainable params: 5,199,247
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 74s 253ms/step - loss: 2.3332 - acc: 0.3649
Epoch 2/10
108/108 [==============================] - 27s 253ms/step - loss: 2.0075 - acc: 0.4016
Epoch 3/10
108/108 [==============================] - 27s 254ms/step - loss: 1.7890 

In [ ]:
# ================== 테스트 파일로 추론해보기 ==================

predict_result = get_predict_result_fn(RESIZED_WIDTH, RESIZED_HEIGHT, model)
get_test_csv_fn(predict_result)

# ============================================================